## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,77.54,70,32,16.04,scattered clouds
1,1,Tuktoyaktuk,CA,69.4541,-133.0374,28.40,100,90,0.00,overcast clouds
2,2,Severo-Kurilsk,RU,50.6789,156.1250,42.71,92,100,5.41,overcast clouds
3,3,Narsaq,GL,60.9167,-46.0500,35.82,67,4,4.03,clear sky
4,4,Lebu,CL,-37.6167,-73.6500,58.12,75,6,15.97,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,77.54,70,32,16.04,scattered clouds
4,4,Lebu,CL,-37.6167,-73.6500,58.12,75,6,15.97,clear sky
5,5,Hermanus,ZA,-34.4187,19.2345,72.81,89,100,8.81,light rain
7,7,Klamath Falls,US,42.2249,-121.7817,53.49,75,20,1.99,few clouds
8,8,Avarua,CK,-21.2078,-159.7750,71.65,83,75,4.61,light rain
9,9,Georgetown,MY,5.4112,100.3354,81.90,90,40,3.00,scattered clouds
10,10,Rikitea,PF,-23.1203,-134.9692,73.29,78,83,6.44,broken clouds
15,15,Presidente Epitacio,BR,-21.7633,-52.1156,87.73,51,95,3.36,overcast clouds
17,17,Ewo,CG,-0.8725,14.8206,72.73,94,92,2.73,overcast clouds
18,18,Ponta Do Sol,PT,32.6667,-17.1000,74.71,67,20,2.51,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                485
City                   485
Country                481
Lat                    485
Lng                    485
Max Temp               485
Humidity               485
Cloudiness             485
Wind Speed             485
Current Description    485
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = preferred_cities_df.dropna()

In [7]:
clean_cities_df.count()

City_ID                481
City                   481
Country                481
Lat                    481
Lng                    481
Max Temp               481
Humidity               481
Cloudiness             481
Wind Speed             481
Current Description    481
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,77.54,scattered clouds,-9.8000,-139.0333,
4,Lebu,CL,58.12,clear sky,-37.6167,-73.6500,
5,Hermanus,ZA,72.81,light rain,-34.4187,19.2345,
7,Klamath Falls,US,53.49,few clouds,42.2249,-121.7817,
8,Avarua,CK,71.65,light rain,-21.2078,-159.7750,
9,Georgetown,MY,81.90,scattered clouds,5.4112,100.3354,
10,Rikitea,PF,73.29,broken clouds,-23.1203,-134.9692,
15,Presidente Epitacio,BR,87.73,overcast clouds,-21.7633,-52.1156,
17,Ewo,CG,72.73,overcast clouds,-0.8725,14.8206,
18,Ponta Do Sol,PT,74.71,few clouds,32.6667,-17.1000,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [10]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
     # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,77.54,scattered clouds,-9.8000,-139.0333,Villa Enata
4,Lebu,CL,58.12,clear sky,-37.6167,-73.6500,Hostal Las Lilas
5,Hermanus,ZA,72.81,light rain,-34.4187,19.2345,Misty Waves Boutique Hotel
7,Klamath Falls,US,53.49,few clouds,42.2249,-121.7817,Holiday Inn Express & Suites Klamath Falls Cen...
8,Avarua,CK,71.65,light rain,-21.2078,-159.7750,Paradise Inn
...,...,...,...,...,...,...,...
681,Guerrero Negro,MX,75.76,clear sky,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
682,Taoudenni,ML,91.98,broken clouds,22.6783,-3.9836,
683,Hoi An,VN,76.96,overcast clouds,15.8794,108.3350,Belle Maison Hadana Hoi An managed by H&K Hosp...
684,Yoichi,JP,53.22,broken clouds,43.2039,140.7703,Marumatta Farm 21


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,77.54,scattered clouds,-9.8000,-139.0333,Villa Enata
4,Lebu,CL,58.12,clear sky,-37.6167,-73.6500,Hostal Las Lilas
5,Hermanus,ZA,72.81,light rain,-34.4187,19.2345,Misty Waves Boutique Hotel
7,Klamath Falls,US,53.49,few clouds,42.2249,-121.7817,Holiday Inn Express & Suites Klamath Falls Cen...
8,Avarua,CK,71.65,light rain,-21.2078,-159.7750,Paradise Inn
...,...,...,...,...,...,...,...
680,Swedru,GH,82.31,scattered clouds,5.5371,-0.6998,Emma-Ike Guesthouse
681,Guerrero Negro,MX,75.76,clear sky,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
683,Hoi An,VN,76.96,overcast clouds,15.8794,108.3350,Belle Maison Hadana Hoi An managed by H&K Hosp...
684,Yoichi,JP,53.22,broken clouds,43.2039,140.7703,Marumatta Farm 21


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))